# NCO mode amplitude test

# Test Description
This script will test the amplitude for waveform in NCO single mode.
The NCO mode internally generated a sine wave with the frequency and phase.
1) In this script test the amplitude accuracy CFR of 400MHz at sampling rate 2500MHz for all channels of the proteus module.

# Test Equipment
1. Tabor Proteus 9484M
2. Keysight Oscilloscope MSO9254A
3. 4 SMA to BNC Cable


# Hardware Connection

    Proteus Module    --->     Oscilloscope
    
    Channel 1         --->      Channel 1
    Channel 2         --->      Channel 2
    Channel 3         --->      Channel 3
    Channel 4         --->      Channel 4
   

# Test Procedure
1. Connect all the channels of Proteus module with the respective channel of oscilloscope.
2. Run the script and observe output on scope.

Proteus release- Aurora

In [1]:
ip_address = '192.90.70.22'
channb =1

In [2]:
## Import required Libraries
import os
import sys
import tempfile
import webbrowser
srcpath = os.path.realpath('SourceFiles')
sys.path.append(srcpath)
from teproteus import TEProteusAdmin as TepAdmin
from teproteus import TEProteusInst as TepInst
from teproteus_functions_v3 import connect
from teproteus_functions_v3 import disconnect
from teproteus_functions_v3 import set_lib_dir_path
from teproteus_functions_v3 import get_cpatured_header
from teproteus_functions_v3 import gauss_env
from teproteus_functions_v3 import iq_kernel
from teproteus_functions_v3 import pack_kernel_data
from teproteus import TEProteusAdmin, TEProteusInst
from tevisainst import TEVisaInst
# matplotlib notebook
import numpy as np
import time
import ipywidgets as widgets
from IPython.core.debugger import set_trace
from scipy.signal import chirp, sweep_poly
import matplotlib.pyplot as plt

plt.style.use('ggplot')
from scipy import signal
import math
import pdb

import pyvisa as visa
from pyvisa.errors import Error

# Instrument connection

In [3]:
# Connect to instrument
inst = connect(ip_address)
# Get the instrument's *IDN
resp = inst.send_scpi_query('*IDN?')
print(resp)

# Get the model:
model_name = inst.send_scpi_query('SYST:INF:MODel?')
print('Model: {0} '.format(model_name))

# Get number of channels
resp = inst.send_scpi_query(":INST:CHAN? MAX")
#print("Number of channels: " + resp)
num_channels = int(resp)

# Get the DAC mode (8 bits or 16 bits)
resp = inst.send_scpi_query(':SYST:INF:DAC?')
if resp == 'M0':
    dac_mode = 16
else:
    dac_mode = 8

print('DAC {0} bits'.format(dac_mode))

if dac_mode == 16:
    max_dac = 65535
    data_type = np.uint16 
else:
    max_dac = 255
    data_type = np.uint8 
    
half_dac = max_dac / 2.0

# Several initializations ..
inst.send_scpi_cmd('*CLS; *RST')
inst.send_scpi_cmd(':INST:CHAN 1')
inst.send_scpi_cmd(':TRAC:DEL:ALL')
resp = inst.send_scpi_query(':SYST:ERR?')
resp = resp.rstrip()
if not resp.startswith('0'):
    print('ERROR: "{0}" after writing binary values'.format(resp))

Service connect
Trying to connect to IP:192.90.70.22
Tabor Electronics,P9484M,000002232771,1.238.6 --Tabor Electronics,P9484M,000000000000,1.238.6 --slot#: 2, slot#: 4, 
Model: P9484M 
DAC 16 bits


# Oscillocope connection

In [4]:
scope_addr2= 'USB0::0x2A8D::0x900E::MY55490134::INSTR'
# connect to scope via USB
try:
    resourceManager = visa.ResourceManager()   # Create a connection (session) to the instrument
    scope = resourceManager.open_resource(scope_addr2)
except visa.Error as ex2:
        print('Couldn\'t connect to \'%s\', exiting now...' % scope_addr2)
        sys.exit()
    ## scope acquisition 
# Send *IDN? and read the response
scope.write('*RST?')
scope.write('*IDN?')
idn = scope.read()
print('*IDN? returned: %s' % idn.rstrip('\n'))

*IDN? returned: KEYSIGHT TECHNOLOGIES,MSO9254A,MY55490134,06.40.01101


# Amplitude test 

In [13]:
test_success = True
sampling_rate = 2500
cfreq = 400e6
volt = [0.5,0.4,0.3,0.25,0.1,0.05]
for channb in np.arange(1,5,1): #loop for all cahnnels
    
    inst.send_scpi_cmd(':INST:CHAN {}'.format(channb))
    channel = inst.send_scpi_query(':INST:CHAN?')
    samp_freq = inst.send_scpi_query(':SOUR:FREQ?')
    inst.send_scpi_cmd(':SOUR:FREQ {}'.format(sampling_rate))
    print('Test begin for channel {0} for CFR={1}MHz at sampling rate {2}MHz'.format(channel,float(cfreq)/1e6,float(samp_freq)/1e6))
    for in_volt in range(len(volt)):
        in_phase = 0
        inst.send_scpi_cmd(':INST:CHAN {}'.format(channb))
        inst.send_scpi_cmd(':MODE NCO')
        resp = inst.send_scpi_query(':MODE?')
        #print('source mode', resp, 'selected')
        #NCO mode , nco frequency and NCO phase
        inst.send_scpi_cmd(':NCO:MODE SING')
        resp = inst.send_scpi_query(":NCO:MODE?") #To check mode
        #print('MODE:',resp)
        inst.send_scpi_cmd(':NCO:CFR1 {}'.format(cfreq))
        inst.send_scpi_cmd(':NCO:PHAS1 0')
        inst.send_scpi_cmd(':VOLT {}'.format(volt[in_volt]))
        #inst.send_scpi_cmd(':NCO:SIXD1 ON')
        inst.send_scpi_cmd(':OUTP ON')
        #scope.write('AUTOscale')
        scope.write(':CHANnel{}:DISPlay ON'.format(channb))
        time.sleep(5)
        
        scope.write('*OPC')
        scope.write('*CLS;:DISPlay:CGRade:LEVels')
        scope.write(':MEASure:CLEar')
        scope.write(':CHANnel{}:SCALe 100E-3'.format(channb))
        scope.write(':TIMebase:SCALe 2E-9')
        scope.write(':MEASure:VPP CHANnel{}'.format(channb))
        time.sleep(2)
        scope.write(':MEASure:RESults?')
        result = scope.read()
        vpp_ch1 = float(result.split(',')[2])
        #print('Peak peak voltage on Channel : ',vpp_ch1)
    #     scope.write('*CLS;:DISPlay:CGRade:LEVels')
    #     scope.write(':MEASure:CLEar')
    #     scope.write(':MEASure:FREQuency CHANnel1')
    #     time.sleep(1)
    #     scope.write(':MEASure:RESults?')
    #     result = scope.read()
    #     frequency = float(result.split(',')[2])
    #     print('Frequency : {}MHz'.format(frequency/1e6))
    #     print("Ocsilloscope frequency should {}MHz".format(cfreq))
        difference =abs(vpp_ch1-volt[in_volt])
        #print('Difference between measured voltage = {}'.format(difference))
        if difference < (0.051):
            print('\tTest pass for {}V '.format(volt[in_volt]))
        else:
            print('\tTest FAIL for {}V'.format(volt[in_volt]))
            test_success = False
        scope.write(':CHANnel{}:DISPlay OFF'.format(channb))
    #input()
if (test_success):
    print('test successed for amplitude')
else:
    print('test failed for amplitude')

Test begin for channel 1 for CFR=400.0MHz at sampling rate 1000.0MHz
Test pass for 0.5V 
Test pass for 0.4V 
Test pass for 0.3V 
Test pass for 0.25V 
Test pass for 0.1V 
Test pass for 0.05V 
Test begin for channel 2 for CFR=400.0MHz at sampling rate 1000.0MHz
Test pass for 0.5V 
Test FAIL for 0.4V
Test FAIL for 0.3V
Test FAIL for 0.25V
Test pass for 0.1V 
Test pass for 0.05V 
Test begin for channel 3 for CFR=400.0MHz at sampling rate 1000.0MHz
Test pass for 0.5V 
Test pass for 0.4V 
Test pass for 0.3V 
Test pass for 0.25V 
Test pass for 0.1V 
Test pass for 0.05V 
Test begin for channel 4 for CFR=400.0MHz at sampling rate 1000.0MHz
Test pass for 0.5V 
Test pass for 0.4V 
Test pass for 0.3V 
Test pass for 0.25V 
Test pass for 0.1V 
Test pass for 0.05V 
test failed for amplitude


# Disconnect

In [ ]:
disconnect()
scope.close()
print('Disconnected')